In [3]:
import plotly.plotly as py
from plotly.graph_objs import *
import pandas as pd
import requests
from pyspark.sql import *
from pyspark.sql.types import StringType
requests.packages.urllib3.disable_warnings()

tbName =["CH37H" ,"LOCMAP_S"]
url = "jdbc:h2:tcp://localhost/~/ssmiSmall"
#url = "jdbc:h2:tcp://10.240.85.53:9292/mem:~/ssmi9095ctest;DB_CLOSE_DELAY=-1"

df1 =sqlContext.read.jdbc(url=url, table = tbName[0],lowerBound = 0, upperBound = 350, numPartitions=200)#dbtable is variable
df1.registerTempTable(tbName[0])
df2 =sqlContext.read.jdbc(url=url, table = tbName[1],lowerBound = 0, upperBound = 350, numPartitions=200) #dbtable is variable
df2.registerTempTable(tbName[1])

#register helper functions for SQL
sqlContext.registerFunction("MONTH", lambda x: str(x)[5:7], StringType())#grab Month
sqlContext.registerFunction("YEAR", lambda x: str(x)[0:4], StringType())
sqlContext.registerFunction("DAY", lambda x: str(x)[8:10], StringType())


In [4]:
rdf1 = sqlContext.sql("SELECT * FROM CH37H")
rdf2 = sqlContext.sql("SELECT * FROM LOCMAP_S")
sqlContext.registerDataFrameAsTable(rdf1, "ssmi")
sqlContext.registerDataFrameAsTable(rdf2, "map")
df1 = sqlContext.sql("SELECT * FROM ssmi t1, map t2 WHERE t1.DATE BETWEEN '1990-01-01' AND '1990-01-31' AND t1.BT < 350 AND t1.LOCID = t2.ID")

In [76]:
#Histogram, ideally, user can select one or more interested timestamp to show one or more histogram for comparison
data = Data([Histogram(x=df1.toPandas()['BT'])])
py.iplot(data, filename="ssmi plot")


In [7]:
#Heatmap for one day, ideally any timestamp/region/or overlap timestamps
dfp=df1.toPandas()

In [12]:
from IPython.display import display
from plotly.graph_objs import Bar, Scatter, Marker, Layout 

display(dfp.head())
display(dfp.tail())

,DATE,LOCID,BT,ID,ROW,COL
0,1990-01-01,31,183,31,0,31
1,1990-01-06,31,0,31,0,31
2,1990-01-31,31,181,31,0,31
3,1990-01-01,231,200,231,0,231
4,1990-01-28,231,281,231,0,231


,DATE,LOCID,BT,ID,ROW,COL
314727,1990-01-02,104630,0,104630,315,50
314728,1990-01-31,104630,178,104630,315,50
314729,1990-01-01,104830,212,104830,315,250
314730,1990-01-16,104830,0,104830,315,250
314731,1990-01-31,104830,194,104830,315,250


In [16]:
#what will be meaningful, selected region, mean BT of selected region over temrpoal range trend
#overlapped bar gragh (can be improved by pre-calculating the mean of BT on each day)
py.iplot([Bar(x=dfp.DATE, y=dfp.BT)], filename='test')

5

In [25]:
#heatmap, mapping hottest/coldest pixels over the temporal range to a single map
from plotly.graph_objs import *
import plotly.tools as tls
dfp.shape
#grab pixels by day
dayId = sqlContext.sql("SELECT DISTINCT DATE FROM ssmi ORDER BY DATE")
dayP = dayId.toPandas()

In [36]:
dayP['DATE'][0:1]

0   1990-01-01
Name: DATE, dtype: datetime64[ns]

In [40]:
day1 = sqlContext.sql("SELECT * FROM ssmi t1, map t2 WHERE t1.DATE BETWEEN '1990-01-01' AND '1990-01-01' AND t1.LOCID = t2.ID ORDER BY t1.LOCID")

In [41]:
day1p = day1.toPandas()

In [46]:
day1p.head()

,DATE,LOCID,BT,ID,ROW,COL
0,1990-01-01,0,201,0,0,0
1,1990-01-01,1,205,1,0,1
2,1990-01-01,2,208,2,0,2
3,1990-01-01,3,201,3,0,3
4,1990-01-01,4,207,4,0,4


In [72]:
import numpy as np
z = day1p.BT.as_matrix()
rown = 316
coln = 332
z = z.reshape(coln, rown)
x = np.arange(1,316,1)
y = np.arange(1,331,1)

In [75]:
trace1 = Surface(
        z=z,
        x=x,
        y=y,
    )
data = Data([trace1])
# Dictionary of style options for all axes
axis = dict(
    showbackground=True, # (!) show axis background
    backgroundcolor="rgb(204, 204, 204)", # set background color to grey
    gridcolor="rgb(255, 255, 255)",       # set grid line color
    zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color
)

# Make a layout object
layout = Layout(
    title='Mapping Pixels in 2D', # set plot title
    scene=Scene(  # (!) axes are part of a 'scene' in 3d plots
        xaxis=XAxis(axis), # set x-axis style
        yaxis=YAxis(axis), # set y-axis style
        zaxis=ZAxis(axis)  # set z-axis style
    )
)
# Make a figure object
fig = Figure(data=data, layout=layout)
# (@) Send to Plotly and show in notebook
py.iplot(fig, filename='Pixels Mapping')